In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Bike sharing demand 
자전거 수요를 RandomForestRegressor 를 사용하여 예측해 봅시다.  
                                                       _jinjae lee_ 만듬


In [ ]:
# data import
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
sampleSubmission = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

train.head()

## 상관관계 분석

In [ ]:
train.corr()

## 결측치 확인

결측치 없음

In [ ]:
train.isnull().sum()

## 데이터 추가

In [ ]:
# datetime 컬럼을 datetime type으로 변경
train['datetime'] = pd.to_datetime(train['datetime'])

# datetime 관련 새로운 feature 생성
train['year'] = train['datetime'].apply(lambda x : x.year)
train['month'] = train['datetime'].apply(lambda x : x.month)
train['day'] = train['datetime'].apply(lambda x : x.day)
train['hour'] = train['datetime'].apply(lambda x : x.hour)

# weather 와 season을  one-hot encording화 
weather=pd.get_dummies(train['weather'],prefix='weather')
train=pd.concat([train,weather],axis=1)

season = pd.get_dummies(train['season'], prefix='season')
train=pd.concat([train, season], axis=1)
train.head()

In [ ]:
# test 데이터 셋에도 동일하게 적용
test['datetime'] = pd.to_datetime(test['datetime'])
test['year'] = test['datetime'].apply(lambda x : x.year)
test['month'] = test['datetime'].apply(lambda x : x.month)
test['day'] = test['datetime'].apply(lambda x : x.day)
test['hour'] = test['datetime'].apply(lambda x : x.hour)

weather=pd.get_dummies(test['weather'],prefix='weather')
test=pd.concat([test,weather],axis=1)

season = pd.get_dummies(test['season'], prefix='season')
test=pd.concat([test, season], axis=1)
test.head()


In [ ]:
train.columns

## 데이터 셋 준비
- Train X에 필요 없는 features를 drop
- train y는 count만 사용
- 여기에서는 특별하게 데이터를 drop 하지 않았지만 상관관계나 그래프를 그려 이상치나 필요하지 않은 feature를 제거하면 성능이 올라갈 것이라고 생각됩니다.

In [ ]:
train_label = train.columns
droped_train_label = train_label.drop(['datetime', 'season', 'weather', 'casual', 'registered', 'count'])
print(droped_train_label)

X = df[droped_train_label]
y = df['count']
print(X.shape)
print(y.shape)


## 모델 생성
- RandomForestRegressor 를 사용
- n_estimators: 생성할 Tree의 개수를 뜻함
- random_state: 동일한 모델을 생성할 시에 같은 결과값을 보여주기 위함

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators= 500, random_state=42)

In [ ]:

rf.fit(X, y)

In [ ]:
# 예측을 해봅시다.
predictions = rf.predict(test_df[droped_train_label])

## 결과 제출

In [ ]:
sampleSubmission['count'] = predictions
display(predictions)
print(sampleSubmission)
sampleSubmission.to_csv('pre0.csv', index=False)